In [1]:
import os
import pandas as pd
import numpy as np
import json
import pprint
import re
import nltk
import csv
import sys

In [2]:
data_dir = "data"
data_folders = os.listdir(data_dir)

for data_folder in data_folders:
    print(data_folder,'\n',os.listdir(os.path.join(data_dir,data_folder)))
    print('-'*30)

empatheticdialogues 
 ['test.csv', 'train.csv', 'valid.csv']
------------------------------
goemotions 
 ['goemotions_1.csv', 'goemotions_2.csv', 'goemotions_3.csv']
------------------------------
processed 
 ['ed_for_goemotions.tsv', 'all_dd_data.tsv', 'all_meld_data.tsv', 'all_names.csv', 'dd_for_goemotions.tsv', 'meld_for_goemotions.tsv', 'dd_for_goemotions.tsv.eval_results.txt', 'dd_for_goemotions.tsv.label_predictions.tsv', 'dd_for_goemotions.tsv.predictions.tsv', 'dd_for_goemotions.tsv.tf_record', 'meld_for_goemotions.tsv.eval_results.txt', 'meld_for_goemotions.tsv.label_predictions.tsv', 'meld_for_goemotions.tsv.predictions.tsv', 'meld_for_goemotions.tsv.tf_record', 'ed_train_clean.tsv', 'ed_test_clean.tsv', 'ed_valid_clean.tsv', 'all_ed_data_train_valid.tsv', 'emotions.csv', 'dd_goemotions_pred_broad.csv', 'dd_goemotions_pred_broad_simple.csv', 'ed_for_goemotions.tsv.eval_results.txt', 'ed_for_goemotions.tsv.label_predictions.tsv', 'ed_for_goemotions.tsv.predictions.tsv', 'ed_f

### Reference for emotion codes

In [3]:
## Use for converting emotion_labels to int
emotion_codes = ("neutral", "anger", "disgust", "fear", "joy", "sadness", "surprise")
emotion_codes = pd.DataFrame(emotion_codes)
emotion_codes.columns = ["broad_emo"]
emotion_codes

,broad_emo
0,neutral
1,anger
2,disgust
3,fear
4,joy
5,sadness
6,surprise


In [4]:
data_folder = '../goemotions/data'
file_name = 'emotions.txt'
emotions = pd.read_csv(os.path.join(data_dir, data_folder, file_name), header=None)
emotions.columns = ["fine_emo"]
emotions["fine_emo_id"] = emotions.index
emotions

,fine_emo,fine_emo_id
0,admiration,0
1,amusement,1
2,anger,2
3,annoyance,3
4,approval,4
5,caring,5
6,confusion,6
7,curiosity,7
8,desire,8
9,disappointment,9


In [5]:
file_name = 'ekman_mapping.json'
with open(os.path.join(data_dir, data_folder, file_name)) as json_data:
    emotions_group = json.load(json_data)
json_data.close()

In [6]:
## add a neutral category
emotions_group['neutral'] = ['neutral']

pprint.pprint(emotions_group)

{'anger': ['anger', 'annoyance', 'disapproval'],
 'disgust': ['disgust'],
 'fear': ['fear', 'nervousness'],
 'joy': ['joy',
         'amusement',
         'approval',
         'excitement',
         'gratitude',
         'love',
         'optimism',
         'relief',
         'pride',
         'admiration',
         'desire',
         'caring'],
 'neutral': ['neutral'],
 'sadness': ['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse'],
 'surprise': ['surprise', 'realization', 'confusion', 'curiosity']}


In [7]:
emotion_codes['fine_emo'] = [emotions_group[x] for x in emotion_codes["broad_emo"]]
emotion_codes

,broad_emo,fine_emo
0,neutral,[neutral]
1,anger,"[anger, annoyance, disapproval]"
2,disgust,[disgust]
3,fear,"[fear, nervousness]"
4,joy,"[joy, amusement, approval, excitement, gratitu..."
5,sadness,"[sadness, disappointment, embarrassment, grief..."
6,surprise,"[surprise, realization, confusion, curiosity]"


In [8]:
broad_emo = list()
broad_emo_id = list()

for x in emotions["fine_emo"]:
    for y in emotion_codes["broad_emo"].index:
        if x in emotion_codes["fine_emo"][y]:
            broad_emo.append(emotion_codes["broad_emo"][y])
            broad_emo_id.append(y)

emotions["broad_emo"] = broad_emo
emotions["broad_emo_id"] = broad_emo_id
emotions

,fine_emo,fine_emo_id,broad_emo,broad_emo_id
0,admiration,0,joy,4
1,amusement,1,joy,4
2,anger,2,anger,1
3,annoyance,3,anger,1
4,approval,4,joy,4
5,caring,5,joy,4
6,confusion,6,surprise,6
7,curiosity,7,surprise,6
8,desire,8,joy,4
9,disappointment,9,sadness,5


In [9]:
file_name = 'sentiment_dict.json'
with open(os.path.join(data_dir, data_folder, file_name)) as json_data:
    sentiment_map = json.load(json_data)
json_data.close()

sentiment_map['neutral'] = ['neutral']
pprint.pprint(sentiment_map)

{'ambiguous': ['realization', 'surprise', 'curiosity', 'confusion'],
 'negative': ['fear',
              'nervousness',
              'remorse',
              'embarrassment',
              'disappointment',
              'sadness',
              'grief',
              'disgust',
              'anger',
              'annoyance',
              'disapproval'],
 'neutral': ['neutral'],
 'positive': ['amusement',
              'excitement',
              'joy',
              'love',
              'desire',
              'optimism',
              'caring',
              'pride',
              'admiration',
              'gratitude',
              'relief',
              'approval']}


In [10]:
sentiment = list()
sentiment_id = list()
sentiment_codes = {"neutral":0, "negative":1, "ambiguous":2, "positive":3}

for x in emotions["fine_emo"]:
    for y in sentiment_map:
        if x in sentiment_map[y]:
            sentiment.append(y)
            sentiment_id.append(sentiment_codes[y])

emotions["sentiment"] = sentiment
emotions["sentiment_id"] = sentiment_id
emotions

,fine_emo,fine_emo_id,broad_emo,broad_emo_id,sentiment,sentiment_id
0,admiration,0,joy,4,positive,3
1,amusement,1,joy,4,positive,3
2,anger,2,anger,1,negative,1
3,annoyance,3,anger,1,negative,1
4,approval,4,joy,4,positive,3
5,caring,5,joy,4,positive,3
6,confusion,6,surprise,6,ambiguous,2
7,curiosity,7,surprise,6,ambiguous,2
8,desire,8,joy,4,positive,3
9,disappointment,9,sadness,5,negative,1


In [11]:
## Save down for reference
emotions.to_csv ('data/processed/emotions.csv', index = False)

### DD Results

#### Alternative way for calculating predicted label - aggregated probabilities of fine emotions

In [12]:
data_folder='processed'
file_name='dd_for_goemotions.tsv.predictions.tsv'
dd_pred_val = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t')
print(dd_pred_val.shape)
dd_pred_val.head()

(102979, 28)


,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,0.008421,0.002836,0.015491,0.028682,0.026227,0.006123,0.003822,0.003060,0.000726,0.007588,...,0.001438,0.000421,0.002497,0.000170,0.008105,0.000333,0.000107,0.002320,0.003833,0.843983
1,0.016101,0.007603,0.013261,0.062777,0.150265,0.069889,0.009169,0.004677,0.020471,0.072058,...,0.006416,0.003752,0.059137,0.000653,0.048835,0.001764,0.002984,0.061557,0.007307,0.247472
2,0.003365,0.003176,0.013244,0.022699,0.007831,0.001758,0.151211,0.515388,0.000249,0.003738,...,0.000427,0.000281,0.000989,0.000068,0.005600,0.000111,0.000143,0.000953,0.019455,0.231153
3,0.004971,0.005370,0.023524,0.063208,0.028742,0.006266,0.245157,0.281368,0.001495,0.023215,...,0.001809,0.001191,0.004716,0.000162,0.022103,0.000310,0.000828,0.006334,0.023354,0.183128
4,0.030738,0.007216,0.009307,0.023914,0.072346,0.022674,0.003322,0.002641,0.002573,0.010938,...,0.003503,0.000696,0.011256,0.000389,0.013479,0.001123,0.000331,0.004381,0.005954,0.721967


In [13]:
dd_pred_broad = pd.DataFrame(dd_pred_val.index.tolist())

## Get the probabilities corresponding to the broader emotion categories
for x in range(len(emotion_codes)):
    
    dd_pred_broad[x] = 0
    
    for y in emotion_codes["fine_emo"][x]:
        dd_pred_broad[x] = dd_pred_broad[x] + dd_pred_val[y]

## Validate sum
print("All sum to 1:", len(dd_pred_broad.sum(axis=1)==1))

## Predicted Label
dd_pred_broad["highest"] = dd_pred_broad.idxmax(axis=1)

dd_pred_broad

All sum to 1: 102979


,0,1,2,3,4,5,6,highest
0,0.843983,0.061078,0.008367,0.003435,0.052924,0.011393,0.018820,0
1,0.247472,0.114430,0.030841,0.035125,0.355362,0.146781,0.069989,4
2,0.231153,0.045513,0.003713,0.002111,0.019837,0.006018,0.691654,6
3,0.183128,0.132540,0.010841,0.008636,0.058267,0.034606,0.571982,6
4,0.721967,0.047144,0.010716,0.005532,0.171639,0.017607,0.025396,0
...,...,...,...,...,...,...,...,...
102974,0.007775,0.004271,0.000554,0.000362,0.973245,0.001539,0.012255,4
102975,0.287620,0.057314,0.004088,0.001931,0.016665,0.005807,0.626575,6
102976,0.263409,0.069850,0.019412,0.026178,0.461288,0.079209,0.080654,4
102977,0.153266,0.056244,0.016916,0.030530,0.532905,0.161733,0.048406,4


In [14]:
## Save down for reference
dd_pred_broad.to_csv ('data/processed/dd_goemotions_pred_broad.tsv', sep='\t', header = True, index = False)

In [15]:
file_name='dd_for_goemotions.tsv'
dd_data = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t', header=None, quoting=csv.QUOTE_NONE)
dd_data.columns = ["utterance", "label", "id"]
print(dd_data.shape)
dd_data.head()

(102979, 3)


,utterance,label,id
0,The kitchen stinks.,2,1_1
1,I 'll throw out the garbage.,0,1_2
2,"So [NAME], how about getting some coffee for t...",4,2_1
3,Coffee? I don't honestly like that kind of stuff.,2,2_2
4,"Come on, you can at least try a little, beside...",0,2_3


In [16]:
dd_data["pred_broad_id"] = dd_pred_broad["highest"]
dd_data.head(10)

,utterance,label,id,pred_broad_id
0,The kitchen stinks.,2,1_1,0
1,I 'll throw out the garbage.,0,1_2,4
2,"So [NAME], how about getting some coffee for t...",4,2_1,6
3,Coffee? I don't honestly like that kind of stuff.,2,2_2,6
4,"Come on, you can at least try a little, beside...",0,2_3,0
5,What's wrong with that? Cigarette is the thing...,1,2_4,6
6,"Not for me, [NAME].",0,2_5,0
7,Are things still going badly with your housegu...,0,3_1,0
8,Getting worse. Now he's eating me out of house...,1,3_2,1
9,"[NAME], I really think you're beating around t...",0,3_3,4


In [17]:
dd_data["correct_pred"] = dd_data.label == dd_data.pred_broad_id

## Overall accuracy of the predictions
dd_data["correct_pred"].value_counts(True)

False    0.58409
True     0.41591
Name: correct_pred, dtype: float64

In [18]:
## Calculate accuracy of the goemotions model applied on DD data for broad emotion labelling
acc_by_emotion = list()

for x in range(len(emotion_codes)):
    
    ## Get all the data for each class
    label_class = dd_data[dd_data.label == x]
    pred_class = dd_data[dd_data.pred_broad_id == x]
    
    ## Calculate recall by emotion group
    label_count = len(label_class)
    correct = len(label_class[label_class.correct_pred == True])
    recall = correct / label_count
    
    ## Calculate precision by emotion group
    pred_count = len(pred_class)
    correct = len(pred_class[pred_class.correct_pred == True])
    ## if class not predicted, then precision is 0
    if correct == 0:
        precision = 0
    else:
        precision = correct / pred_count
    
    ## Calculate F1 score
    ## if class not predicted, F1 is 0
    if precision == 0:
        f1 = 0
    else:
        f1 = 2 * (recall * precision) / (recall + precision)
    
    acc_by_emotion.append([x, emotion_codes.broad_emo[x], label_count, pred_count, recall, precision, f1])

acc_by_emotion = pd.DataFrame(acc_by_emotion)
acc_by_emotion.columns = ["broad_emotion_group", "broad_emotion", "label_count", "prediction_count", 
                          "recall", "precision", "F1"]
acc_by_emotion

,broad_emotion_group,broad_emotion,label_count,prediction_count,recall,precision,F1
0,0,neutral,85572,35553,0.374795,0.902090,0.529569
1,1,anger,1022,2790,0.150685,0.055197,0.080797
2,2,disgust,353,0,0.000000,0.000000,0.000000
3,3,fear,174,0,0.000000,0.000000,0.000000
4,4,joy,12885,30602,0.692355,0.291517,0.410284
5,5,sadness,1150,2881,0.457391,0.182575,0.260977
6,6,surprise,1823,31153,0.634668,0.037139,0.070172


#### Simple way for calculating predicted label

In [19]:
data_folder='processed'
file_name='dd_for_goemotions.tsv.label_predictions.tsv'
dd_pred = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t')
print(dd_pred.shape)
dd_pred.head()

(102979, 7)


,text,emotion_1,prob_1,emotion_2,prob_2,emotion_3,prob_3
0,The kitchen stinks.,neutral,0.8440,NaN,NaN,NaN,NaN
1,I 'll throw out the garbage.,neutral,0.2475,approval,0.1503,disappointment,0.0721
2,"So [NAME], how about getting some coffee for t...",curiosity,0.5154,neutral,0.2312,confusion,0.1512
3,Coffee? I don't honestly like that kind of stuff.,curiosity,0.2814,confusion,0.2452,neutral,0.1831
4,"Come on, you can at least try a little, beside...",neutral,0.7220,approval,0.0723,NaN,NaN


In [20]:
file_name='dd_for_goemotions.tsv'
dd_data = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t', header=None, quoting=csv.QUOTE_NONE)
dd_data.columns = ["utterance", "label", "id"]

dd_data["pred"] = dd_pred.emotion_1
dd_data.head()

,utterance,label,id,pred
0,The kitchen stinks.,2,1_1,neutral
1,I 'll throw out the garbage.,0,1_2,neutral
2,"So [NAME], how about getting some coffee for t...",4,2_1,curiosity
3,Coffee? I don't honestly like that kind of stuff.,2,2_2,curiosity
4,"Come on, you can at least try a little, beside...",0,2_3,neutral


In [21]:
## Get the pred labels for broad emotions
broad_emo_id = list()
for x in range(len(dd_data)):
    y = int(emotions.broad_emo_id[emotions["fine_emo"] == dd_data["pred"][x]])
    broad_emo_id.append(y)
    
dd_data["pred_broad_id"] = broad_emo_id
dd_data.head(10)

,utterance,label,id,pred,pred_broad_id
0,The kitchen stinks.,2,1_1,neutral,0
1,I 'll throw out the garbage.,0,1_2,neutral,0
2,"So [NAME], how about getting some coffee for t...",4,2_1,curiosity,6
3,Coffee? I don't honestly like that kind of stuff.,2,2_2,curiosity,6
4,"Come on, you can at least try a little, beside...",0,2_3,neutral,0
5,What's wrong with that? Cigarette is the thing...,1,2_4,curiosity,6
6,"Not for me, [NAME].",0,2_5,neutral,0
7,Are things still going badly with your housegu...,0,3_1,neutral,0
8,Getting worse. Now he's eating me out of house...,1,3_2,neutral,0
9,"[NAME], I really think you're beating around t...",0,3_3,admiration,4


In [22]:
## Save down for reference
dd_data.to_csv ('data/processed/dd_goemotions_pred_broad_simple.tsv', sep='\t', header = True, index = False)

In [23]:
dd_data["correct_pred"] = dd_data.label == dd_data.pred_broad_id

## Overall accuracy of the predictions
dd_data["correct_pred"].value_counts(True)

True     0.545694
False    0.454306
Name: correct_pred, dtype: float64

In [24]:
## Calculate accuracy of the goemotions model applied on DD data for broad emotion labelling
acc_by_emotion = list()

for x in range(len(emotion_codes)):
    
    ## Get all the data for each class
    label_class = dd_data[dd_data.label == x]
    pred_class = dd_data[dd_data.pred_broad_id == x]
    
    ## Calculate recall by emotion group
    label_count = len(label_class)
    correct = len(label_class[label_class.correct_pred == True])
    recall = correct / label_count
    
    ## Calculate precision by emotion group
    pred_count = len(pred_class)
    correct = len(pred_class[pred_class.correct_pred == True])
    ## if class not predicted, then precision is 0
    if correct == 0:
        precision = 0
    else:
        precision = correct / pred_count
    
    ## Calculate F1 score
    ## if class not predicted, F1 is 0
    if precision == 0:
        f1 = 0
    else:
        f1 = 2 * (recall * precision) / (recall + precision)
    
    acc_by_emotion.append([x, emotion_codes.broad_emo[x], label_count, pred_count, recall, precision, f1])

acc_by_emotion = pd.DataFrame(acc_by_emotion)
acc_by_emotion.columns = ["broad_emotion_group", "broad_emotion", "label_count", "prediction_count", 
                          "recall", "precision", "F1"]
acc_by_emotion

,broad_emotion_group,broad_emotion,label_count,prediction_count,recall,precision,F1
0,0,neutral,85572,52196,0.544711,0.893019,0.676674
1,1,anger,1022,163,0.038160,0.239264,0.065823
2,2,disgust,353,0,0.000000,0.000000,0.000000
3,3,fear,174,0,0.000000,0.000000,0.000000
4,4,joy,12885,21431,0.621653,0.373758,0.466838
5,5,sadness,1150,2565,0.433043,0.194152,0.268102
6,6,surprise,1823,26624,0.568294,0.038912,0.072837


### MELD Results

In [25]:
data_folder='processed'
file_name='meld_for_goemotions.tsv.label_predictions.tsv'
meld_pred = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t')
print(meld_pred.shape)
meld_pred.head()

(3719, 7)


,text,emotion_1,prob_1,emotion_2,prob_2,emotion_3,prob_3
0,"Oh my God, he's lost it. He's totally lost it.",neutral,0.1865,sadness,0.1477,disappointment,0.1047
1,What?,curiosity,0.4777,neutral,0.2868,confusion,0.1426
2,"Or! Or, we could go to the bank, close our acc...",neutral,0.8870,NaN,NaN,NaN,NaN
3,You're a genius!,admiration,0.9010,NaN,NaN,NaN,NaN
4,"Aww, man, now we won't be bank buddies!",neutral,0.2620,approval,0.2318,realization,0.0570


In [26]:
file_name='meld_for_goemotions.tsv'
meld_data = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t', header=None, quoting=csv.QUOTE_NONE)
meld_data.columns = ["utterance", "label", "id"]

meld_data["pred"] = meld_pred.emotion_1
meld_data.head()

,utterance,label,id,pred
0,"Oh my God, he's lost it. He's totally lost it.",5,Phoebe_0_0,neutral
1,What?,6,Monica_0_1,curiosity
2,"Or! Or, we could go to the bank, close our acc...",0,Ross_1_0,neutral
3,You're a genius!,4,Chandler_1_1,admiration
4,"Aww, man, now we won't be bank buddies!",5,Joey_1_2,neutral


In [27]:
## Get the pred labels for broad emotions
broad_emo_id = list()
for x in range(len(meld_data)):
    y = int(emotions.broad_emo_id[emotions["fine_emo"] == meld_data["pred"][x]])
    broad_emo_id.append(y)
    
meld_data["pred_broad_id"] = broad_emo_id
meld_data.head(10)

,utterance,label,id,pred,pred_broad_id
0,"Oh my God, he's lost it. He's totally lost it.",5,Phoebe_0_0,neutral,0
1,What?,6,Monica_0_1,curiosity,6
2,"Or! Or, we could go to the bank, close our acc...",0,Ross_1_0,neutral,0
3,You're a genius!,4,Chandler_1_1,admiration,4
4,"Aww, man, now we won't be bank buddies!",5,Joey_1_2,neutral,0
5,"Now, there's two reasons.",0,Chandler_1_3,neutral,0
6,Hey.,0,Phoebe_1_4,neutral,0
7,Hey!,4,All_1_5,neutral,0
8,"Ohh, you guys, remember that cute client I tol...",0,Phoebe_1_6,curiosity,6
9,Where?!,6,Rachel_1_7,neutral,0


In [28]:
## Save down for reference
meld_data.to_csv ('data/processed/meld_goemotions_pred_broad_simple.tsv', sep='\t', header = True, index = False)

In [29]:
meld_data["correct_pred"] = meld_data.label == meld_data.pred_broad_id

## Overall accuracy of the predictions
meld_data["correct_pred"].value_counts(True)

False    0.566012
True     0.433988
Name: correct_pred, dtype: float64

In [30]:
## Calculate accuracy of the goemotions model applied on DD data for broad emotion labelling
acc_by_emotion = list()

for x in range(len(emotion_codes)):
    
    ## Get all the data for each class
    label_class = meld_data[meld_data.label == x]
    pred_class = meld_data[meld_data.pred_broad_id == x]
    
    ## Calculate recall by emotion group
    label_count = len(label_class)
    correct = len(label_class[label_class.correct_pred == True])
    recall = correct / label_count
    
    ## Calculate precision by emotion group
    pred_count = len(pred_class)
    correct = len(pred_class[pred_class.correct_pred == True])
    ## if class not predicted, then precision is 0
    if correct == 0:
        precision = 0
    else:
        precision = correct / pred_count
    
    ## Calculate F1 score
    ## if class not predicted, F1 is 0
    if precision == 0:
        f1 = 0
    else:
        f1 = 2 * (recall * precision) / (recall + precision)
    
    acc_by_emotion.append([x, emotion_codes.broad_emo[x], label_count, pred_count, recall, precision, f1])

acc_by_emotion = pd.DataFrame(acc_by_emotion)
acc_by_emotion.columns = ["broad_emotion_group", "broad_emotion", "label_count", "prediction_count", 
                          "recall", "precision", "F1"]
acc_by_emotion

,broad_emotion_group,broad_emotion,label_count,prediction_count,recall,precision,F1
0,0,neutral,1726,2283,0.664542,0.502409,0.572213
1,1,anger,498,17,0.020080,0.588235,0.038835
2,2,disgust,90,0,0.000000,0.000000,0.000000
3,3,fear,90,0,0.000000,0.000000,0.000000
4,4,joy,565,577,0.366372,0.358752,0.362522
5,5,sadness,319,98,0.128527,0.418367,0.196643
6,6,surprise,431,744,0.484919,0.280914,0.355745


### ED Results

In [31]:
data_folder='processed'
file_name='ed_for_goemotions.tsv.label_predictions.tsv'
ed_pred = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t')
print(ed_pred.shape)
ed_pred.head()

(86084, 7)


,text,emotion_1,prob_1,emotion_2,prob_2,emotion_3,prob_3
0,I remember going to see the fireworks with my ...,admiration,0.2694,approval,0.2135,neutral,0.1787
1,"Was this a friend you were in love with, or ju...",love,0.3503,curiosity,0.2529,neutral,0.1153
2,This was a best friend. I miss her.,approval,0.2201,neutral,0.1893,admiration,0.1841
3,Where has she gone?,curiosity,0.3872,neutral,0.3797,confusion,0.1280
4,We no longer talk.,neutral,0.6205,annoyance,0.0976,disapproval,0.0805


In [32]:
file_name='ed_for_goemotions.tsv'
ed_data = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t', header=None, quoting=csv.QUOTE_NONE)
ed_data.columns = ["utterance", "label", "id"]
print(ed_data.shape)

ed_data["pred"] = ed_pred.emotion_1
ed_data.head()

(96247, 3)


,utterance,label,id,pred
0,I remember going to see the fireworks with my ...,0,1_1,admiration
1,"Was this a friend you were in love with, or ju...",0,1_2,love
2,This was a best friend. I miss her.,0,1_3,approval
3,Where has she gone?,0,1_4,curiosity
4,We no longer talk.,0,1_5,neutral


In [33]:
ed_data.pred.value_counts()

neutral       35190
admiration    11376
curiosity      9441
approval       4851
optimism       4553
sadness        4224
joy            4180
amusement      2851
love           2213
surprise       1945
gratitude      1936
remorse        1740
desire          710
anger           463
confusion       250
excitement      122
annoyance        39
Name: pred, dtype: int64

In [34]:
## Get the pred labels for broad emotions
broad_emo_id = list()

## Encountering some issues, doing section by section
for x in range(len(ed_data)):
    y = int(emotions.broad_emo_id[emotions["fine_emo"] == ed_data["pred"][x]])
    broad_emo_id.append(y)
    
ed_data["pred_broad_id"] = broad_emo_id
ed_data.head(10)

KeyboardInterrupt: 

In [ ]:
## Save down for reference
ed_data.to_csv ('data/processed/ed_goemotions_pred_broad_simple.tsv', sep='\t', header = True, index = False)